# Visualiser lingvistiske oppgaver

In [1]:
from pathlib import Path 
import pandas as pd
import numpy as np
from utils import model_groups

paths = (Path("../contrastive/results"), Path("../greedy/results"))
results = {"model": [], "task": [], "accuracy": [], "generation_strategy": []}

for p in paths:
    for e in p.iterdir():
        df = pd.read_json(e, lines=True)
        df = df[df.dataset.isin(["analogies", "idioms", "synonyms"])]
        accuracies = []
        for x in df.itertuples():
            results["model"].append(e.stem)
            results["accuracy"].append(x.results[0]["accuracy"])
            results["task"].append(x.dataset)
            results["generation_strategy"].append(p.parent.name)
        
            accuracies.append(x.results[0]["accuracy"])
        
        results["model"].append(e.stem)
        results["accuracy"].append(np.mean(accuracies))
        results["task"].append("all (average)")
        results["generation_strategy"].append(p.parent.name)

results_df = pd.DataFrame(results)

In [2]:
def get_order(model_group:str) -> dict[str, str]:
    match model_group:
        case "base_scratch_and_deltas":
            return {"model": ['mimir-mistral-7b-base-scratch','mimir-7b-factual','mimir-7b-nonfiction', 'mimir-7b-newspapers','mimir-7b-translated', 
                              'mimir-7b-untranslated', 'mimir-7b-untranslated-withnewspapers', 'mimir-7b-books','mimir-7b-rightholders', 'mimir-7b-fiction']}
        case "base_and_extended":
            return {"model": ['mimir-mistral-7b-base-scratch', 'mimir-mistral-7b-extended-scratch', 'mimir-mistral-7b-base', 'mimir-mistral-7b-extended']}
        case "bases_and_intructs":
            return {"model": [
        'mimir-mistral-7b-base-scratch', 'mimir-mistral-7b-base-scratch-instruct',
        'mimir-mistral-7b-extended-scratch', 'mimir-mistral-7b-extended-scratch-instruct',
        'mimir-mistral-7b-base', 'mimir-mistral-7b-base-instruct',
        'mimir-mistral-7b-extended', 'mimir-mistral-7b-extended-instruct',
    ]}

In [3]:
from dash import Dash, html, dcc, callback, Output, Input
import plotly.express as px

df = results_df
app = Dash()

app.layout = [
    html.H1(children='Linguistic tasks', style={'textAlign':'center', "color": "White"}),
    dcc.Dropdown(list(model_groups.keys()), 'base_scratch_and_deltas', id='dropdown-modelgroup'),
    dcc.Dropdown(df.task.unique(), 'all (average)', id='dropdown-task'),

    dcc.Graph(id='graph-content')
]

@callback(
    Output('graph-content', 'figure'),
    [Input('dropdown-modelgroup', 'value'),
    Input('dropdown-task', 'value')]
)
def update_graph(selected_modelgroup, selected_task):
    dff = df[df.model.isin(model_groups[selected_modelgroup])]
    dff = dff[dff.task == selected_task].sort_values("accuracy")
    return px.bar(dff, x='model', y='accuracy', color='generation_strategy', barmode="group")#, category_orders=get_order(selected_modelgroup))

if __name__ == '__main__':
    app.run(debug=True)
